In [ ]:
_int    = list(range(1000))              #from 0 to 999
_float  = [0.001*i for i in range(1001)] #from 0.0 to 1.0
_num    = ['_int', '_float']
_list   = ['_list_of_int', '_list_of_float', '_list_of_num', '_complex_list']
_bool   = [True, False]
_var    = ['_int', '_float', '_num', '_list', '_bool']


_math_op       = ['+', '-', '*', '**', '/', '//', '%']
_comparison_op = ['==', '!=', '<>', '>', '<', '>=', '<=']
_assignment_op = ['=', '+=', '-=', '*=', '/=', '//=']
_bitwise_op    = ['&', '|', '^', '~', '<<', '>>']
_membership_op = ['in', 'not in']
_identity_op   = ['is', 'is not']


_while_OP = ['_comparison_op', '_membership_op', '_identity_op']
_if_OP    = ['_comparison_op', '_membership_op', '_identity_op']


for_loop = \
[
'for i in _list',
'for n in range(_int)',
'for n,i in enumerate(_list)'
]


while_loop = \
[
'while _bool',
'while _num _while_OP _num'
]


if_stmt = \
[
'if _var _membership_op _list',
'if _var _comparison_op _var',
'elif _var _membership_op _list',
'elif _var _comparison_op _var',
'else'
]


flow_ctrl = \
[
'break',
'continue',
'pass'
]


blocks = 3
max_indent = 2

for _ in range(blocks):
    block = ''
    for n in range(max_indent):
        indent = '\t'*n

In [1]:
import random

def gen_code(complexity=3, nest_lvl=2):
    '''
    Generates a code template made by 'complexity' blocks.
    Each block will be formed by 'nest_lvl' nested loops/if-stmt.
    
    Example:
    >>> a = gen_code(1,2)
        for line in a.split('\n'):
            print(line)
    
    >>> for variable in range(variable):
            do_nothing()
            for variable in iterable:
                do_something(variable)
    '''
    
    def replace(string, word, replacers):
        while word in string:
            string = string.replace(word, random.choice(replacers), 1)
        return string  

    if complexity:
        if type(complexity) == type(nest_lvl) == type(1):
            pass
        else:
            raise ValueError('complexity and nest_lvl must be positive integers.')
    else:
        raise ValueError('complexity must be greater than 0.')
    
    
    variables            = ['_x','_y','_z','_n','_i','_j',]
    iterables            = ['_string','_list', '_tuple', '_dict']
    math_operators       = ['+','-','*','**','/','//','%']
    comparison_operators = ['==','!=','<>','>','<','>=','<=']
    assignment_operators = ['=','+=','-=','*=','/=','//=']
    bitwise_operators    = ['&','|','^','~','<<','>>']
    membership_operators = ['in','not in']
    identity_operators   = ['is','is not']
    
    constructs = ['for variable in iterable:',
                  'for variable in range(variable):',
                  'for variable in range(len(iterable)):',

                  'while variable comparison_operators variable:',
                  'while variable membership_operators iterable:',

                  'if variable comparison_operators variable:',
                  'if variable membership_operators iterable:',
                  'if variable identity_operators variable:',

                  'elif variable comparison_operators variable:',
                  'elif variable membership_operators iterable:',
                  'elif variable identity_operators variable:',

                  'else:']

    functions = ['_f1(variable)',
                 '_f2(iterable)',
                 '_f3()',
                 'results.append(variable)']



    block = []
    begin  = [item for item in constructs[0:8]]
    middle = [item for item in constructs[8:]]

    for n in range(complexity):
        if nest_lvl == 0:
            block.append(random.sample(functions,1)[0])
            
        else:
            for lvl in range(nest_lvl):
                ind = lvl*'\t'
                block.append(ind + random.sample(begin,1)[0]) #first_line
                for m in range(random.randint(1,complexity)):
                    block.append('{}{}'.format(ind + '\t',*random.sample(functions,1)))
                ind += '\t'
        

    code = ''''''
    for item in block:
        item = replace(item, 'variable', variables)
        item = replace(item, 'iterable', iterables)
        item = replace(item, 'math_operators', math_operators)
        item = replace(item, 'comparison_operators', comparison_operators)
        item = replace(item, 'assignment_operators', assignment_operators)
        item = replace(item, 'bitwise_operators', bitwise_operators)
        item = replace(item, 'membership_operators', membership_operators)
        item = replace(item, 'bitwise_operators', assignment_operators)
        item = replace(item, 'identity_operators', identity_operators)
        code += item + '\n'
    
    
    return code

In [3]:
from alefuncs import Timeout
import time

with Timeout(1):
    print(gen_code(1,2))
    print(gen_code(2,1))
    time.sleep(2)

if _z not in _string:
	_f1(_n)
	for _y in range(_z):
		_f1(_z)

if _i in _dict:
	_f1(_i)
while _x < _x:
	_f3()
	_f1(_i)



TimeoutError: Timeout